In [67]:
from bs4 import BeautifulSoup
import requests
import time
import datetime
import pandas as pd
import re

In [84]:
price= []
mkmodel = []
mileage= []
year = []

In [41]:
url = "https://www.cars.com/shopping/results/?page=1&page_size=100&list_price_max=&makes[]=&maximum_distance=20&models[]=&stock_type=used&zip=53151"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",
           "Accept-Encoding":"gzip, deflate",  "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
           "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

In [83]:
def pagelooper(n,m):
    for i in range(n,m):
        while(1):
            try:
                page = requests.get(f"""https://www.cars.com/shopping/results/?page={i}&page_size=100&list_price_max=&makes[]
                =&maximum_distance=20&models[]=&stock_type=used&zip=53151""",headers=headers)

                soup1 = BeautifulSoup(page.content,"html.parser")
                soup2= BeautifulSoup(soup1.prettify(),'html.parser')
                search = soup2.find_all(class_="primary-price")
                search2 = soup2.find_all(class_="title")
                search3 = soup2.find_all(class_='mileage')
            except AttributeError:
                print('Retrying')
                time.sleep(3)
            else:
                break
        for i,j,k in zip(search,search2,search3,):
            price.append(i.get_text().strip())
            year.append(j.get_text().strip()[0:4])
            mkmodel.append(j.get_text().strip()[5:])
            mileage.append(k.get_text().strip())
        time.sleep(6)
        v +=1
        print(f"Page {v} scanned...")

In [85]:
pagelooper(1,40)

In [6]:
import datetime
today = datetime.date.today()
print(today)

2021-09-09


In [86]:
cardata= list(zip(price,year,mkmodel,mileage))

In [50]:
cardata

[('$23,598', '2013', ' Lexus RX 450h Base', '69,570 mi.'),
 ('$26,192', '2019', ' Mazda CX-5 Touring', '8,847 mi.'),
 ('$45,303', '2019', ' Nissan Armada SL', '31,818 mi.'),
 ('$14,311', '2016', ' Mazda Mazda3 i Sport', '75,312 mi.'),
 ('$49,990', '2018', ' BMW M2 Base', '14,684 mi.'),
 ('$68,750', '2019', ' Audi Q8 3.0T Premium', '20,629 mi.'),
 ('$27,987', '2018', ' Lincoln MKC Premiere', '34,362 mi.'),
 ('$27,603', '2018', ' Lincoln MKC Premiere', '45,961 mi.'),
 ('$44,992', '2020', ' Mercedes-Benz GLB 250 Base 4MATIC', '12,504 mi.'),
 ('$48,995', '2021', ' Hyundai Palisade Limited', '13,109 mi.'),
 ('$31,985', '2018', ' Alfa Romeo Stelvio Base', '35,277 mi.'),
 ('$32,498', '2019', ' Ford Explorer XLT', '30,526 mi.'),
 ('$35,990', '2018', ' Mercedes-Benz C-Class C 300 4MATIC', '12,388 mi.'),
 ('$33,750', '2019', ' INFINITI QX50 ESSENTIAL', '52,435 mi.'),
 ('$9,497', '2010', ' Toyota Camry SE', '88,593 mi.'),
 ('$16,787', '2013', ' Chevrolet Camaro 1LT', '91,950 mi.'),
 ('$47,995', '

In [87]:
need = pd.DataFrame(cardata,columns=['price','year','mkmodel','mileage'])

In [100]:
need

,price,year,mkmodel,mileage
0,"$23,598",2013,Lexus RX 450h Base,"69,570 mi."
1,"$26,192",2019,Mazda CX-5 Touring,"8,847 mi."
2,"$45,303",2019,Nissan Armada SL,"31,818 mi."
3,"$14,311",2016,Mazda Mazda3 i Sport,"75,312 mi."
4,"$49,990",2018,BMW M2 Base,"14,684 mi."
...,...,...,...,...
3879,"$18,762",2014,Ford Explorer XLT,"122,217 mi."
3880,"$34,872",2018,Ford Flex SEL,"28,575 mi."
3881,"$14,490",2014,Nissan Rogue SL,"146,501 mi."
3882,"$33,999",2019,Volkswagen Golf Alltrack TSI SEL,"15,033 mi."


In [101]:
need.to_csv('carscom.csv')

In [59]:
getcount = requests.get("""https://www.cars.com/shopping/results/?page=1&page_size=100&list_price_max=&makes[]
                =&maximum_distance=20&models[]=&stock_type=used&zip=53151""",headers=headers)
setup1 = BeautifulSoup(page.content,"html.parser")
setup2 = BeautifulSoup(soup1.prettify(),'html.parser')
pagecounter = setup2.find(class_="total-filter-count").get_text()

In [82]:
x =  ''.join(re.findall(r"\d+", pagecounter.strip()))

In [80]:
int(x)//100


95

In [91]:
pd.ExcelWriter('Cars.com used info.xlsx').save()

,price,year,mkmodel,mileage
0,"$23,598",2013,Lexus RX 450h Base,"69,570 mi."
1,"$26,192",2019,Mazda CX-5 Touring,"8,847 mi."
2,"$45,303",2019,Nissan Armada SL,"31,818 mi."
3,"$14,311",2016,Mazda Mazda3 i Sport,"75,312 mi."
4,"$49,990",2018,BMW M2 Base,"14,684 mi."
...,...,...,...,...
3879,"$18,762",2014,Ford Explorer XLT,"122,217 mi."
3880,"$34,872",2018,Ford Flex SEL,"28,575 mi."
3881,"$14,490",2014,Nissan Rogue SL,"146,501 mi."
3882,"$33,999",2019,Volkswagen Golf Alltrack TSI SEL,"15,033 mi."
